<a href="https://colab.research.google.com/github/cristianbossolasco/Inflacion-Dolar-Argentina/blob/main/Historico_Dolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import json

In [2]:
def get_historico_dolar(tipo, fecha_inicial, fecha_final):
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
  }
  response = requests.get(f'https://mercados.ambito.com/dolar/{tipo}/grafico/{fecha_inicial}/{fecha_final}', headers=headers)

  data  = response.content

  # Decodificar la cadena de bytes y convertirla en una lista
  decoded_data = data.decode('utf-8')
  data_list = json.loads(decoded_data)

  # Crear el dataframe
  df = pd.DataFrame(data_list[1:], columns=data_list[0])

  # Me aseguro que la fecha tenga formato fecha
  df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')

  return df

In [3]:
fecha_desde, fecha_hasta = '2000-01-01', '2024-12-31'

df_dolar_oficial = get_historico_dolar('formal', fecha_desde, fecha_hasta)
df_dolar_blue = get_historico_dolar('informal', fecha_desde, fecha_hasta)

# elimino duplicados
df_dolar_oficial_sin_dupl = pd.DataFrame(df_dolar_oficial.groupby('fecha')['Dólar Oficial'].max()).sort_index()
df_dolar_blue_sin_dupl = pd.DataFrame(df_dolar_blue.groupby('fecha')['Dólar Informal'].max()).sort_index()

# Utiliza el método resample para obtener el último valor de cada mes
df_dolar_oficial_mes = df_dolar_oficial_sin_dupl.resample('M').last()
df_dolar_blue_mes = df_dolar_blue_sin_dupl.resample('M').last()

# unifico los dataframe
df_dolar = df_dolar_oficial_mes.merge(
              df_dolar_blue_mes,
              left_on='fecha',
              right_on='fecha'
          )
df_dolar.rename(columns={'Dólar Oficial': 'Oficial', 'Dólar Informal' : 'Blue'}, inplace=True)
df_dolar.tail(10)

<ipython-input-3-c4964778c17f>:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_dolar_oficial_mes = df_dolar_oficial_sin_dupl.resample('M').last()
<ipython-input-3-c4964778c17f>:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_dolar_blue_mes = df_dolar_blue_sin_dupl.resample('M').last()


,Oficial,Blue
fecha,,
2024-02-29,888.70,1045.0
2024-03-31,901.63,1010.0
2024-04-30,920.57,1040.0
2024-05-31,934.94,1225.0
2024-06-30,950.78,1365.0
2024-07-31,971.53,1370.0
2024-08-31,991.42,1305.0
2024-09-30,1004.94,1235.0
2024-10-31,1020.75,1180.0
